<a href="https://colab.research.google.com/github/ayush139/colllab/blob/main/NLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [88]:
from google.colab import files
  
  
uploaded = files.upload()

Saving tr_dt_manual.xlsx to tr_dt_manual (1).xlsx


In [89]:

import pandas as pd
import io
  
df = pd.read_excel(io.BytesIO(uploaded['tr_dt_manual.xlsx']))
print(df)

       index  ...     Label
0        0.0  ...  negative
1        1.0  ...  positive
2        2.0  ...  negative
3        3.0  ...   neutral
4        4.0  ...   neutral
...      ...  ...       ...
10747    NaN  ...  negative
10748    NaN  ...  negative
10749    NaN  ...  negative
10750    NaN  ...  negative
10751    NaN  ...  negative

[10752 rows x 6 columns]


In [90]:
data = df[['Corpus','Label']]
print(data)

                                                  Corpus     Label
0                       billing time take extremely long  negative
1      staff try her best to stand along to process t...  positive
2      however waste of time for involve during such ...  negative
3                      give time space more variety need   neutral
4      just want to recommend that you can think to g...   neutral
...                                                  ...       ...
10747                        billing time extremely high  negative
10748  but when try to utilize the same for purchase ...  negative
10749               make charge too high compare to gold  negative
10750                               labour cost too high  negative
10751            price more high compare to market price  negative

[10752 rows x 2 columns]


In [96]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Corpus'].values)
X = tokenizer.texts_to_sequences(data['Corpus'].values)
X = pad_sequences(X)

embed_dim = 542
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(3,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 542, 542)          2710000   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 542, 542)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 196)               579376    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 12        
Total params: 3,289,979
Trainable params: 3,289,979
Non-trainable params: 0
_________________________________________________________________
None


In [97]:

Y = pd.get_dummies(data['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(7203, 542) (7203, 3)
(3549, 542) (3549, 3)


In [98]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size)

Epoch 1/7
226/226 [==============================] - 1302s 6s/step - loss: 0.8951 - accuracy: 0.5377
Epoch 2/7
226/226 [==============================] - 1300s 6s/step - loss: 0.5819 - accuracy: 0.7630
Epoch 3/7
226/226 [==============================] - 1309s 6s/step - loss: 0.4688 - accuracy: 0.8435
Epoch 4/7
226/226 [==============================] - 1324s 6s/step - loss: 0.3755 - accuracy: 0.8753
Epoch 5/7
226/226 [==============================] - 1311s 6s/step - loss: 0.3114 - accuracy: 0.8989
Epoch 6/7
226/226 [==============================] - 1315s 6s/step - loss: 0.2612 - accuracy: 0.9120
Epoch 7/7
226/226 [==============================] - 1311s 6s/step - loss: 0.2332 - accuracy: 0.9177


In [99]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [100]:
# create on Colab directory
model.save('LSTM.h5')    
model_file = drive.CreateFile({'title' : 'LSTM.h5'})
model_file.SetContentFile('LSTM.h5')
model_file.Upload()

In [101]:

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1nAZBGQJpjrjlVsJmkmhNoe5vkS_orp3Z'})

In [102]:
sentiment = model.predict(X_test,batch_size=1)

In [103]:
sentiment

array([[5.9303520e-03, 7.1468875e-02, 9.2260075e-01],
       [5.7926998e-03, 3.6367450e-03, 9.9057055e-01],
       [1.6289525e-02, 8.8960350e-01, 9.4106935e-02],
       ...,
       [1.0831836e-03, 9.5474869e-01, 4.4168185e-02],
       [4.9153528e-01, 4.1758552e-01, 9.0879098e-02],
       [4.0635292e-05, 1.2162416e-05, 9.9994719e-01]], dtype=float32)

In [104]:
pred=pd.DataFrame(sentiment,columns=["Negative","Neutral","Positive"])

In [105]:
pred['Max']=pred.max(axis = 1)

In [106]:
pred.head()

,Negative,Neutral,Positive,Max
0,0.005930,0.071469,0.922601,0.922601
1,0.005793,0.003637,0.990571,0.990571
2,0.016290,0.889603,0.094107,0.889603
3,0.010435,0.906063,0.083502,0.906063
4,0.467587,0.489232,0.043181,0.489232


In [107]:
for i in range(0,len(pred)):
  if (pred['Negative'][i]==pred['Max'][i]):
    pred['Negative'][i]=1
    pred['Neutral'][i]=0
    pred['Positive'][i]=0
  elif (pred['Neutral'][i]==pred['Max'][i]):
    pred['Negative'][i]=0
    pred['Neutral'][i]=1
    pred['Positive'][i]=0
  else:    
    pred['Negative'][i]=0
    pred['Neutral'][i]=0
    pred['Positive'][i]=1

In [108]:
pred.head()

,Negative,Neutral,Positive,Max
0,0.0,0.0,1.0,0.922601
1,0.0,0.0,1.0,0.990571
2,0.0,1.0,0.0,0.889603
3,0.0,1.0,0.0,0.906063
4,0.0,1.0,0.0,0.489232


In [109]:
pred.drop(['Max'],axis=1,inplace=True)

In [110]:
ytest=pd.DataFrame(Y_test,columns=["Negative","Neutral","Positive"])

In [111]:
pred.head()

,Negative,Neutral,Positive
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0


In [112]:
ytest.head()

,Negative,Neutral,Positive
0,0,1,0
1,0,0,1
2,0,1,0
3,0,1,0
4,0,1,0


In [36]:
from google.colab import files
pred.to_csv('pred.csv') 
files.download('pred.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
from google.colab import files
  
  
uploaded = files.upload()


Saving pred.csv to pred (1).csv


In [41]:
pred = pd.read_csv(io.BytesIO(uploaded['pred.csv']))
print(pred)

      Negative  Neutral  Positive
0            0        1         0
1            0        0         1
2            0        1         0
3            0        1         0
4            0        1         0
...        ...      ...       ...
3544         1        0         0
3545         0        0         1
3546         0        1         0
3547         0        1         0
3548         0        0         1

[3549 rows x 3 columns]


In [113]:
from sklearn.metrics import classification_report
print(classification_report(ytest, pred,target_names=ytest.columns))

              precision    recall  f1-score   support

    Negative       0.72      0.81      0.76       979
     Neutral       0.74      0.69      0.71      1137
    Positive       0.91      0.88      0.89      1433

   micro avg       0.80      0.80      0.80      3549
   macro avg       0.79      0.79      0.79      3549
weighted avg       0.80      0.80      0.80      3549
 samples avg       0.80      0.80      0.80      3549



In [114]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

111/111 - 84s - loss: 0.6382 - accuracy: 0.7991
score: 0.64
acc: 0.80
